## The following is an example showing both a subquery SELECT & JOIN SELECT that return the same result set and execution plan

In [6]:
-- USE AdventureWorks2019;
-- GO

-- /* SELECT statement built using a subquery. */
-- SELECT [Name]
-- FROM Production.Product
-- WHERE ListPrice =
--     (SELECT ListPrice
--      FROM Production.Product
--      WHERE [Name] = 'Chainring Bolts' );
-- GO

-- /* SELECT statement built using a join that returns
--    the same result set. */
-- SELECT Prd1.[Name]
-- FROM Production.Product AS Prd1
--      JOIN Production.Product AS Prd2
--        ON (Prd1.ListPrice = Prd2.ListPrice)
-- WHERE Prd2.[Name] = 'Chainring Bolts';
-- GO

Commands completed successfully.

Total execution time: 00:00:00.063

In the following example, the BusinessEntityID column in the WHERE clause of the outer query is

implicitly qualified by the table name in the outer query FROM clause (Sales.Store). The reference

to CustomerID in the select list of the subquery is qualified by the subquery FROM clause, that is, by

the Sales.Customer table.

In [5]:
-- USE AdventureWorks2019;
-- GO
-- SELECT [Name]
-- FROM Sales.Store
-- WHERE BusinessEntityID NOT IN
-- (SELECT CustomerID
-- FROM Sales.Customer
-- WHERE TerritoryID = 5);
-- GO

Commands completed successfully.

Total execution time: 00:00:00.063

# Multiple levels of nesting:

**The following query finds the names of employees who are also sales persons.**

- The innermost query returns the sales person IDs. 

- The query at the next higher level is evaluated with these sales person IDs and returns the contact ID numbers of the employees. 

- Finally, the outer query uses the contact IDs to find the names of the employees.

In [7]:
USE AdventureWorks2019;
GO
SELECT LastName, FirstName
FROM Person.Person
WHERE BusinessEntityID IN
(SELECT BusinessEntityID
FROM HumanResources.Employee
WHERE BusinessEntityID IN
(SELECT BusinessEntityID
FROM Sales.SalesPerson)
);

Commands completed successfully.

(17 rows affected)

Total execution time: 00:00:00.264

LastName,FirstName
Jiang,Stephen
Blythe,Michael
Mitchell,Linda
Carson,Jillian
Vargas,Garrett
Reiter,Tsvi
Ansman-Wolfe,Pamela
Ito,Shu
Saraiva,José
Campbell,David


## You can also express this query as a join:

In [8]:
USE AdventureWorks2019;
GO
SELECT LastName, FirstName
FROM Person.Person c
INNER JOIN HumanResources.Employee e
ON c.BusinessEntityID = e.BusinessEntityID
JOIN Sales.SalesPerson s
ON e.BusinessEntityID = s.BusinessEntityID;
GO

Commands completed successfully.

(17 rows affected)

Total execution time: 00:00:00.144

LastName,FirstName
Abbas,Syed
Campbell,David
Vargas,Garrett
Reiter,Tsvi
Carson,Jillian
Blythe,Michael
Valdez,Rachel
Alberts,Amy
Mitchell,Linda
Saraiva,José


## Correlated subqueries

Many queries can be evaluated by executing the subquery once and substituting the resulting value or values into the WHERE clause of the outer query. In queries that include a correlated subquery (also known as a repeating subquery), the subquery depends on the outer query for its values. This means that the subquery is executed repeatedly, once for each row that might be selected by the outer query. This query retrieves one instance of each employee's first and last name for which the bonus in the SalesPerson table is 5000 and for which the employee identification numbers match in the Employee and SalesPerson tables.

In [9]:
USE AdventureWorks2019;
GO
SELECT DISTINCT c.LastName, c.FirstName, e.BusinessEntityID 
FROM Person.Person AS c JOIN HumanResources.Employee AS e
ON e.BusinessEntityID = c.BusinessEntityID 
WHERE 5000.00 IN
    (SELECT Bonus
    FROM Sales.SalesPerson sp
    WHERE e.BusinessEntityID = sp.BusinessEntityID) ;
GO

Commands completed successfully.

(2 rows affected)

Total execution time: 00:00:00.157

LastName,FirstName,BusinessEntityID
Ansman-Wolfe,Pamela,280
Saraiva,José,282


## Subqueries with table aliases

Many statements in which the subquery and the outer query refer to the same table can be stated as self-joins (joining a table to itself). For example, you can find addresses of employees from a particular state using a subquery:

In [10]:
USE AdventureWorks2019;
GO
SELECT StateProvinceID, AddressID
FROM Person.Address
WHERE AddressID IN
    (SELECT AddressID
     FROM Person.Address
     WHERE StateProvinceID = 39);
GO

--SELF JOIN

USE AdventureWorks2019;
GO
SELECT e1.StateProvinceID, e1.AddressID
FROM Person.Address AS e1
INNER JOIN Person.Address AS e2
ON e1.AddressID = e2.AddressID
AND e2.StateProvinceID = 39;
GO


--Table aliases e1 and e2 are required because the table being joined to itself appears in two different roles. 
--Aliases can also be used in nested queries that refer to the same table in an inner and outer query.

USE AdventureWorks2019;
GO
SELECT e1.StateProvinceID, e1.AddressID
FROM Person.Address AS e1
WHERE e1.AddressID IN
    (SELECT e2.AddressID
     FROM Person.Address AS e2
     WHERE e2.StateProvinceID = 39);
GO

Commands completed successfully.

(4 rows affected)

Commands completed successfully.

(4 rows affected)

Commands completed successfully.

(4 rows affected)

Total execution time: 00:00:00.386

StateProvinceID,AddressID
39,942
39,955
39,972
39,22660


StateProvinceID,AddressID
39,942
39,955
39,972
39,22660


StateProvinceID,AddressID
39,942
39,955
39,972
39,22660


## Subqueries with IN

The result of a subquery introduced with IN (or with NOT IN) is a list of zero or more values. After the subquery returns results, the outer query makes use of them. The following query finds the names of all the wheel products that Adventure Works Cycles makes.

  
This statement is evaluated in two steps. First, the inner query returns the subcategory identification number that matches the name 'Wheel' (17). Second, this value is substituted into the outer query, which finds the product names that go with the subcategory identification numbers in Production.Product.

In [14]:
USE AdventureWorks2019;
GO
SELECT [Name]
FROM Production.Product
WHERE ProductSubcategoryID IN
    (SELECT ProductSubcategoryID
     FROM Production.ProductSubcategory
     WHERE [Name] = 'Wheels');
GO


-- USE AdventureWorks2019;
-- GO
-- SELECT [Name]
-- FROM Production.Product
-- WHERE ProductSubcategoryID IN ('17');
-- GO


--One difference in using a join rather than a subquery for this and similar problems is that the join lets you show columns from more than one table in the result. 
--For example, if you want to include the name of the product subcategory in the result, you must use a join version.

USE AdventureWorks2019;
GO
SELECT p.[Name], s.[Name]
FROM Production.Product p
INNER JOIN Production.ProductSubcategory s
ON p.ProductSubcategoryID = s.ProductSubcategoryID
AND s.[Name] = 'Wheels';
GO

Commands completed successfully.

(14 rows affected)

Commands completed successfully.

(14 rows affected)

Total execution time: 00:00:00.286

Name
LL Mountain Front Wheel
ML Mountain Front Wheel
HL Mountain Front Wheel
LL Road Front Wheel
ML Road Front Wheel
HL Road Front Wheel
Touring Front Wheel
LL Mountain Rear Wheel
ML Mountain Rear Wheel
HL Mountain Rear Wheel


Name,Name
LL Mountain Front Wheel,Wheels
ML Mountain Front Wheel,Wheels
HL Mountain Front Wheel,Wheels
LL Road Front Wheel,Wheels
ML Road Front Wheel,Wheels
HL Road Front Wheel,Wheels
Touring Front Wheel,Wheels
LL Mountain Rear Wheel,Wheels
ML Mountain Rear Wheel,Wheels
HL Mountain Rear Wheel,Wheels


**The following query finds the name of all vendors whose credit rating is good, from whom Adventure Works Cycles orders at least 20 items, and whose average lead time to deliver is less than 16 days.**

In [15]:
USE AdventureWorks2019;
GO
SELECT [Name]
FROM Purchasing.Vendor
WHERE CreditRating = 1
AND BusinessEntityID IN
    (SELECT BusinessEntityID
     FROM Purchasing.ProductVendor
     WHERE MinOrderQty >= 20
     AND AverageLeadTime < 16);
GO

---USING JOIN TO EXPRESS THE SAME STATEMENT
USE AdventureWorks2019;
GO
SELECT DISTINCT [Name]
FROM Purchasing.Vendor v
INNER JOIN Purchasing.ProductVendor p
ON v.BusinessEntityID = p.BusinessEntityID
WHERE CreditRating = 1
  AND MinOrderQty >= 20
  AND AverageLeadTime < 16;
GO

Commands completed successfully.

(13 rows affected)

Total execution time: 00:00:00.143

Name
"Compete Enterprises, Inc"
International Trek Center
First National Sport Co.
Comfort Road Bicycles
Circuit Cycles
First Rate Bicycles
Jeff's Sporting Goods
Competition Bike Training Systems
Electronic Bike Repair & Supplies
Crowley Sport


## **Subqueries with NOT IN**

Subqueries introduced with the keyword NOT IN also return a list of zero or more values.
The following query finds the names of the products that are not finished bicycles.

This statement cannot be converted to a join. The analogous not-equal join has a different meaning: It finds the names of products that are in some subcategory that is not a finished bicycle.

In [17]:
-- USE AdventureWorks2019;
-- GO
-- SELECT [Name]
-- FROM Production.Product
-- WHERE ProductSubcategoryID NOT IN
--     (SELECT ProductSubcategoryID
--      FROM Production.ProductSubcategory
--      WHERE [Name] = 'Mountain Bikes' 
--         OR [Name] = 'Road Bikes'
--         OR [Name] = 'Touring Bikes');
-- GO

Commands completed successfully.

Total execution time: 00:00:00.061

### Subqueries in UPDATE, DELETE, and INSERT statements

**Subqueries can be nested in the UPDATE, DELETE, INSERT and SELECT data manipulation (DML) statements.**

The following example doubles the value in the ListPrice column in the Production.Product table. The subquery in the WHERE clause references the Purchasing.ProductVendor table to restrict the rows updated in the Product table to just those supplied by BusinessEntity 1540.

In [21]:
-- USE AdventureWorks2019;
-- GO 
-- UPDATE Production.Product
-- SET ListPrice = ListPrice * 2
-- WHERE ProductID IN
--     (SELECT ProductID 
--      FROM Purchasing.ProductVendor
--      WHERE BusinessEntityID = 1540);
-- GO

-- UPDATE Production.Product
-- SET ListPrice = ListPrice * 2
-- FROM Production.Product AS p
-- INNER JOIN Purchasing.ProductVendor AS pv
--     ON p.ProductID = pv.ProductID AND BusinessEntityID = 1540;
-- GO   


--NOT WORKING-ERROR MESSAGE

Commands completed successfully.

Total execution time: 00:00:00.067

## Subqueries with comparison operators

In [23]:
-- USE AdventureWorks2019;
-- GO
-- SELECT CustomerID
-- FROM Sales.Customer
-- WHERE TerritoryID =
--     (SELECT TerritoryID
--      FROM Sales.SalesPerson
--      WHERE BusinessEntityID = 276);
-- GO

Commands completed successfully.

Total execution time: 00:00:00.062

# GROUP BY-HAVING

In [25]:
-- USE AdventureWorks2019;
-- GO
-- SELECT [Name]
-- FROM Production.Product
-- WHERE ListPrice >
--     (SELECT MIN (ListPrice)
--      FROM Production.Product
--      GROUP BY ProductSubcategoryID
--      HAVING ProductSubcategoryID = 14);
-- GO

Commands completed successfully.

Total execution time: 00:00:00.063

## Comparison operators modified by ANY, SOME, or ALL

**The =ANY operator is equivalent to IN. For example, to find the names of all the wheel products that Adventure Works Cycles makes, you can use either IN or =ANY.**


The <>ANY operator, however, differs from NOT IN:

<>ANY means not = a, or not = b, or not = c
NOT IN means not = a, and not = b, and not = c
<>ALL means the same as NOT IN

In [28]:
--comparison operator modified by ANY. 
--It finds the products whose list prices are greater than or equal to the maximum list price of any product subcategory.

-- USE AdventureWorks2019;
-- GO
-- SELECT [Name]
-- FROM Production.Product
-- WHERE ListPrice >= ANY
--     (SELECT MAX (ListPrice)
--      FROM Production.Product
--      GROUP BY ProductSubcategoryID);
-- GO

--Using =ANY
USE AdventureWorks2019;
GO
SELECT [Name]
FROM Production.Product
WHERE ProductSubcategoryID =ANY
    (SELECT ProductSubcategoryID
     FROM Production.ProductSubcategory
     WHERE Name = 'Wheels');
GO

--Using IN
USE AdventureWorks2019;
GO
SELECT [Name]
FROM Production.Product
WHERE ProductSubcategoryID IN
    (SELECT ProductSubcategoryID
     FROM Production.ProductSubcategory
     WHERE Name = 'Wheels');
GO

-- the following query finds customers located in a territory not covered by any sales persons.
USE AdventureWorks2019;
GO
SELECT CustomerID
FROM Sales.Customer
WHERE TerritoryID <> ANY
    (SELECT TerritoryID
     FROM Sales.SalesPerson);
GO

Commands completed successfully.

(14 rows affected)

Commands completed successfully.

(14 rows affected)

Total execution time: 00:00:00.262

Name
LL Mountain Front Wheel
ML Mountain Front Wheel
HL Mountain Front Wheel
LL Road Front Wheel
ML Road Front Wheel
HL Road Front Wheel
Touring Front Wheel
LL Mountain Rear Wheel
ML Mountain Rear Wheel
HL Mountain Rear Wheel


Name
LL Mountain Front Wheel
ML Mountain Front Wheel
HL Mountain Front Wheel
LL Road Front Wheel
ML Road Front Wheel
HL Road Front Wheel
Touring Front Wheel
LL Mountain Rear Wheel
ML Mountain Rear Wheel
HL Mountain Rear Wheel


## Subqueries with EXISTS

When a subquery is introduced with the keyword EXISTS, the subquery functions as an existence test. The WHERE clause of the outer query tests whether the rows that are returned by the subquery exist. The subquery does not actually produce any data; it returns a value of TRUE or FALSE.

A subquery introduced with EXISTS has the following syntax:

WHERE [NOT] EXISTS (subquery)

In [29]:
USE AdventureWorks2019;
GO
SELECT [Name]
FROM Production.Product
WHERE EXISTS
    (SELECT * 
     FROM Production.ProductSubcategory
     WHERE ProductSubcategoryID = 
            Production.Product.ProductSubcategoryID
        AND [Name] = 'Wheels');
GO

--USING IN INSTEAD OF EXISTS
-- USE AdventureWorks2019;
-- GO
-- SELECT [Name]
-- FROM Production.Product
-- WHERE ProductSubcategoryID IN
--     (SELECT ProductSubcategoryID
--      FROM Production.ProductSubcategory
--      WHERE [Name] = 'Wheels');
-- GO

Commands completed successfully.

(14 rows affected)

Total execution time: 00:00:00.138

Name
LL Mountain Front Wheel
ML Mountain Front Wheel
HL Mountain Front Wheel
LL Road Front Wheel
ML Road Front Wheel
HL Road Front Wheel
Touring Front Wheel
LL Mountain Rear Wheel
ML Mountain Rear Wheel
HL Mountain Rear Wheel


## Subqueries with NOT EXISTS

NOT EXISTS works like EXISTS, except the WHERE clause in which it is used is satisfied if no rows are returned by the subquery.

For example, to find the names of products that are not in the wheels subcategory:

In [31]:
-- USE AdventureWorks2019;
-- GO
-- SELECT [Name]
-- FROM Production.Product
-- WHERE NOT EXISTS
--     (SELECT * 
--      FROM Production.ProductSubcategory
--      WHERE ProductSubcategoryID = 
--             Production.Product.ProductSubcategoryID
--         AND [Name] = 'Wheels');
-- GO

Commands completed successfully.

Total execution time: 00:00:00.063

## Subqueries used in place of an expression

In Transact-SQL, a subquery can be substituted anywhere an expression can be used in SELECT, UPDATE, INSERT, and DELETE statements, except in an ORDER BY list.

The following example illustrates how you might use this enhancement. This query finds the prices of all mountain bike products, their average price, and the difference between the price of each mountain bike and the average price.

In [32]:
USE AdventureWorks2019;
GO
SELECT [Name], ListPrice, 
(SELECT AVG(ListPrice) FROM Production.Product) AS Average, 
    ListPrice - (SELECT AVG(ListPrice) FROM Production.Product)
    AS Difference
FROM Production.Product
WHERE ProductSubcategoryID = 1;
GO

Commands completed successfully.

(32 rows affected)

Total execution time: 00:00:00.263

Name,ListPrice,Average,Difference
"Mountain-100 Silver, 38",3399.99,438.6662,2961.3238
"Mountain-100 Silver, 42",3399.99,438.6662,2961.3238
"Mountain-100 Silver, 44",3399.99,438.6662,2961.3238
"Mountain-100 Silver, 48",3399.99,438.6662,2961.3238
"Mountain-100 Black, 38",3374.99,438.6662,2936.3238
"Mountain-100 Black, 42",3374.99,438.6662,2936.3238
"Mountain-100 Black, 44",3374.99,438.6662,2936.3238
"Mountain-100 Black, 48",3374.99,438.6662,2936.3238
"Mountain-200 Silver, 38",2319.99,438.6662,1881.3238
"Mountain-200 Silver, 42",2319.99,438.6662,1881.3238


In [2]:
-- CREATE TABLE charms_class (
--     student_name varchar(255),
--     charms_grade varchar(255)
-- );
-- INSERT INTO charms_class (student_name, charms_grade)
-- VALUES
--     ('Harry', 'C'),
--     ('Ron', 'D'),
--     ('Hermione', 'A'),
--     ('Luna', 'B'),
--     ('Neville', 'B')
-- ;
-- CREATE TABLE potions_class (
--     student_name varchar(255),
--     potions_grade varchar(255)
-- );
-- INSERT INTO potions_class (student_name, potions_grade)
-- VALUES 
--     ('Harry', 'A'),
--     ('Ron', 'C'),
--     ('Hermione', 'B'),
--     ('Ginny', 'A'),
--     ('Draco', 'D')
;

Commands completed successfully.

Total execution time: 00:00:00.064

In [ ]:
-- --INNER JOIN
-- select * 
-- from 
-- charms_class INNER JOIN potions_class 
-- on 
-- charms_class.student_name=potions_class.student_name;


-- --OUTER JOIN
-- select * 
-- from 
-- charms_class FULL OUTER JOIN potions_class 
-- on 
-- charms_class.student_name=potions_class.student_name;


-- --LEFT OUTER JOIN
-- select * 
-- from 
-- charms_class LEFT OUTER JOIN potions_class 
-- on 
-- charms_class.student_name=potions_class.student_name;


-- --RIGHT OUTER JOIN
-- select * 
-- from 
-- charms_class RIGHT OUTER JOIN potions_class 
-- on 
-- charms_class.student_name=potions_class.student_name;